## Create test and training folders for initial CNN

In [13]:
import numpy as np
import pandas as pd

import os
from shutil import copyfile


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [14]:
#watch_df = pd.read_csv('all_watch_info.csv')
watch_df = pd.read_csv('all_watch_info.csv')

In [15]:
#watch_df

Lets start something that *seems* simple ... whether the watch is on a braclet or strap

In [16]:
X = watch_df[['image_name','gender']]
y = watch_df['band_simple']
#y = watch_df['shape_simple']

In [17]:
# 0 = braclet
# 1 = strap

y = LabelEncoder().fit_transform(y)

Uncomment this code 

In [18]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0, random_state=42)

In [19]:
X_train = X
y_train = y

Build Directories

In [20]:
# feature = 'band_data'
# positive = 'strap'
# null = 'bracelet'

feature = 'f_vec_data_padded'
positive = 'strap'
null = 'bracelet'

In [21]:
def copy_train_test(feature,positive,null):

    # Build folders
    try:
        os.stat(feature)
    except:
        os.mkdir(feature) 

    # Train Data Paths
    try:
        os.stat(feature + '/train/')
    except:
        os.mkdir(feature + '/train/')   

    try:
        os.stat(feature + '/train/'+positive)
    except:
        os.mkdir(feature + '/train/'+positive)     


    try:
        os.stat(feature + '/train/'+null)
    except:
        os.mkdir(feature + '/train/'+null)      


    # Test Data Paths  
    try:
        os.stat(feature + '/test/')
    except:
        os.mkdir(feature + '/test/')   

    try:
        os.stat(feature + '/test/'+positive)
    except:
        os.mkdir(feature + '/test/'+positive)     


    try:
        os.stat(feature + '/test/'+null)
    except:
        os.mkdir(feature + '/test/'+null)      



    # Make Copies for Train Data
    for ix, val in enumerate(y_train):

        if X_train.iloc[ix,:]['gender'] == 'mens':
            f_path = 'prestige_time_pics_mens_padded/'
        else:
            f_path = 'prestige_time_pics_womens_padded/'

        f_name = X_train.iloc[ix,:]['image_name'] + '.jpg'


        if val == 1:
            f_out = feature+'/train/'+positive+'/'+f_name
        else:
            f_out = feature+'/train/'+null+'/'+f_name

        copyfile(f_path+f_name,f_out)


    # Make copies for test data    
    for ix, val in enumerate(y_test):

        if X_test.iloc[ix,:]['gender'] == 'mens':
            f_path = 'prestige_time_pics_mens_padded/'
        else:
            f_path = 'prestige_time_pics_womens_padded/'

        f_name = X_test.iloc[ix,:]['image_name'] + '.jpg'


        if val == 1:
            f_out = feature+'/test/'+positive+'/'+f_name
        else:
            f_out = feature+'/test/'+null+'/'+f_name

        copyfile(f_path+f_name,f_out)

In [22]:
copy_train_test(feature,positive,null)